# 2. Application Setup - Creating a Boilerplate QA App

This section guides you through creating a functional question-answering application using the PhariaAI applications quick start guide. You'll leverage the document collection you set up in the previous section and transform it into an interactive Q&A tool that can respond to natural language queries.

The Q&A application we'll build acts as the user-facing layer of your RAG system, connecting the document index with large language models to provide contextually relevant answers based on your specific document collection.

## Application Components

The PhariaAI Q&A application architecture consists of several interconnected elements:

- **Pharia CLI**: Command-line interface for scaffolding, creating, and managing applications
- **Configuration Files**: Define how your application accesses document collections and language models
- **User Interface**: Provides a web-based interface for users to submit questions and receive answers
- **Service Layer**: Handles request processing and orchestrates the retrieval and generation workflow
- **Skill Component**: Implements the core RAG logic that retrieves documents and generates answers

## What You'll Learn

In this section, you'll learn how to:

1. Set up a new Q&A application using the Pharia CLI with the specialized qa template
2. Configure the application components to access your document collection
3. Customize the application settings for optimal retrieval and response performance
4. Deploy your application for local testing and production environments

## Prerequisites

Before starting, ensure you have the following in your Pharia Setup:

- **PhariaAI Version**: You are running PhariaAI version 0.52.4 or higher
- **Custom Skills Support**: Enabled in your PhariaAI instance with a default kernel namespace named app created and you have the credentials for the OCI registry
- **Custom Applications Support**: Enabled in your PhariaAI instance and you have the credentials for the OCI registry
- **Platform Components**: You have enabled PhariaAssistant, PhariaOS Manager, PhariaKernel, and PhariaStudio and you have the URLs to access them

As a User make sure to have:

- **API Token**: A valid Aleph Alpha API token with appropriate permissions
- **Components URLS**: You have the URLs to access PhariaAssistant, PhariaKernel, and PhariaStudio
- **Permissions**: StudioUser permission as described in the User Setup section

<br>
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
<br>

### 1. Creating a New PhariaAI Application

Our first step is to create the application scaffold using the PhariaAI CLI. This tool automatically generates the necessary files and folder structure for a complete application. When we create a new application, the CLI will generate three main components: a user interface, a backend service, and a skill that contains the core application logic.

The generated application includes the following:
* a PhariaAI application UI
* a PhariaAI application service
* a PhariaAI application skill

Run the command below in your terminal:

In [ ]:
npx @aleph-alpha/pharia-ai-cli create

The CLI will prompt you to provide configuration information for your new application. First, it will ask for an application name:

```python
✔ Please provide a new app name › <your-app-name>
```

> Enter the name: "rag_tutorial"

Next, the CLI will ask you to select a template. Templates provide pre-configured application structures optimized for different use cases:

```python
✔ Please select a template › base / summary / qa
```

> For our RAG application, select the **qa** template.

### 2. Setting Up Application Components

After creating the application scaffold, we need to set up each component individually. The application consists of three main parts that operate together: the UI (user interface), the service (backend), and the skill (application logic). Each component needs its own dependencies installed and environment configured.

We'll need to use separate terminal sessions for each component, as they'll be running simultaneously during development.

<br>
- - - - - - - - - - - - - - - - - - - - - - - - - - -
<br>

#### 2.1 PhariaAI Application UI

The UI component provides the web interface that users will interact with. It's built using modern web technologies and needs to be configured with authentication to communicate with the Pharia platform.

> **Note**: To get a token, see: How to obtain a Bearer token in Pharia Studio https://docs.aleph-alpha.com/products/pharia-ai/pharia-studio/tutorial/add-intelligence-layer/#set-your-environment-variables

In your Terminal use the following commands:

In [ ]:
cd <your-app-name>/ui
pnpm install

Before starting the UI, you need to set up authentication. Open the environment file and add your Pharia API token -> Set your Pharia AI user token in `<your-app-name>/ui/.env.` with the identifier `VITE_USER_TOKEN=<your-bearer-token>`

Now you can start the UI development server:

In [ ]:
pnpm run dev

<br>
- - - - - - - - - - - - - - - - - - - - - - - - - - -
<br>

#### 2.2 PhariaAI Application Service

The service component functions as the backend for your application. It handles API requests from the UI, manages application state, and communicates with the skill component to process user queries. It's built with Python and runs as a web server.

In your Terminal use the following commands:

In [ ]:
cd <your-app-name>/service

Before setting up the service, ensure your environment is properly configured:

- Ensure your local Python version matches the one specified in pyproject.toml (e.g., pyenv install x.x.x && pyenv local x.x.x; see https://github.com/pyenv/pyenv)
- Verify that any missing environment variables are set in <your-app-name>/service/.env

Now install the dependencies and start the service:

In [ ]:
uv sync --dev
uv run dev

<br>
- - - - - - - - - - - - - - - - - - - - - - - - - - -
<br>

#### 2.3 PhariaAI Application Skill

The skill component contains the core logic of your application, including document retrieval and answer generation. Unlike the UI and service components, the skill is designed to be executed by the PhariaKernel platform rather than running directly on your local machine. However, we still need to set up the development environment locally.

First, you need to configure the environment variables for the OCI registry, which will be used to publish the skill. This refers to the .env file in the project root:

```bash
SKILL_REGISTRY=registry.your-organization.com
SKILL_REPOSITORY=your-pharia-skills-repository
SKILL_REGISTRY_USER=your-registry-username
SKILL_REGISTRY_TOKEN=your-registry-password
```

Navigate to the skill directory and set up the environment:

In [ ]:
cd <your-app-name>/skill

Load the environment variables and install dependencies:

> **Note**: Ensure your local Python version matches the one specified in pyproject.toml

In [ ]:
set -a && source ../.env
uv sync --dev

> **Note**: Congratulations! You now have your first PhariaAI application up and running. To see your application in action, simply visit the application UI URL. The user interface is already set up to communicate with the application service, and the application service is configured to work with the application skill.

But there are still some steps to go:

> **Warning**: At this point, API calls from the application service to the PhariaAI skill will fail. This is because the skill needs to be served through a PhariaKernel. To resolve this, you'll need to publish the skill so that it can be served through your PhariaKernel instance.

<br>
- - - - - - - - - - - - - - - - - - - - - - - - - - -
<br>

### 3. Publishing the Skill

Although your application is running locally, there's a critical step needed to make it fully functional. Currently, the skill component is only accessible locally and not yet available to the PhariaKernel. To solve this, we need to build and publish the skill to the registry, making it available for production use.


In your terminal make sure you are at the skill folder (`cd <your-app-name>/skill`), start with building the skill. This process packages your code and dependencies into a container that can be deployed on the PhariaKernel:

In [ ]:
cd <your-app-name>/skill
uv run pharia-skill build qa

After successfully building the skill, you are asked to publish it to the registry. This makes the skill accessible to the PhariaKernel service.

You will be asked to provide both a name & a tag for your skill when publishing. For this example lets choose the name ```qa-rag-tutorial```
> **Note**:  Please make sure that the name is not used already to avoid overriding another skill.

How the response will look like:

```bash
╭────────────────────────────────── Publish Successful ──────────────────────────────────╮
│                                                                                        │
│  Skill: qa-rag-tutorial                                                                │
│  Tag: latest                                                                           │
│  Registry: <your-registry>/skills                                                      │
│                                                                                        │
│  Published in 11.14 seconds                                                            │
│                                                                                        │
╰───────────────────────────────────────────────────────────────────────────────────
```

The final step is to configure the PhariaKernel to recognize and serve your skill. After publishing, update your ```namespace.toml``` file (ask your PhariaAI Operator for the location of this file) by adding the new skill entry. Once updated, your PhariaAI application is ready to use the published skill.

---

### 4. Adjusting the Routing Configuration

After publishing your skill, you need to connect your application's service component to the published skill. This requires updating the routing configuration in your service component to point to the correct namespace and skill name.

The service component acts as a bridge between the UI and the skill, forwarding requests and responses between them. By default, the service is configured to use a skill named "qa" in the "app" namespace, but we need to update this to match our published skill.

To be able to connect the frontend to the skill you just published, we need to adjust the `<Application-name>/service/source/routes.py` file to point to the right namespace and skill. In the file you need to update the following line of code:

```python
skill = Skill(namespace="app", name="qa")

In most of our deployments, the namespace in which Kernel is running will be called "playground", and the skill will have the name you gave it. So in our example we need to replace the above line with:

```python
skill = Skill(namespace="playground", name="qa-rag-tutorial")

As we have not adjusted the qa skill yet, it still points to the example collection we set up, which contains papers regarding transformers. While we'll change this in the next tutorial, when previewing your application now you can ask questions like: "What is an encoder?"

> **Restart Service**: After these changes we need to restart the service component to point to the changed namespace & skill!

### 5. Previewing Your Application

With your application components set up, skill published, and routing configured, you can now preview and test your application. The Pharia platform provides tools for previewing applications locally before making them available to all users.

The preview process allows you to test your application in a development environment, ensuring everything is working correctly before deploying to production. During preview, your application runs locally but connects to the published skill through the PhariaKernel platform.

> **Note**: Before using any pharia-ai-cli command below, make sure all necessary environment variables are configured correctly in `<your-app-name>/.env`.

When you're ready to preview your PhariaAI application, run the following command:

In [ ]:
cd <your-app-name>
npx @aleph-alpha/pharia-ai-cli preview

This preview command starts your application in a local development mode and provides a URL that you can add to PhariaAssistant for testing:

1. This command starts the application locally.

2. Copy the preview URL and follow the instructions in How to use developer mode in PhariaAssistant

3. Add the URL to the Developer Mode Server URL list in PhariaAssistant

4. Your application should now appear in PhariaAssistant's left sidebar.

---

In this section, you've successfully created and deployed a PhariaAI Q&A application that can answer questions based on your document collection:

✅ **Created the application scaffold** using the PhariaAI CLI with the qa template, which generated:
   - A responsive user interface for submitting questions
   - A backend service to handle requests
   - A skill component that implements the RAG logic

✅ **Set up and configured all components**:
   - Installed dependencies for each component
   - Set up authentication and environment variables
   - Started each component in development mode

✅ **Published the skill to the registry**:
   - Built the skill container with all necessary dependencies
   - Published it to the OCI registry for use by PhariaKernel
   - Updated your service configuration to point to the published skill

✅ **Previewed and tested the application**:
   - Started the application in preview mode
   - Connected it to PhariaAssistant for testing
   - Verified proper document retrieval and answer generation

Your Q&A application now provides a complete interface for users to ask questions about your document collection and receive contextually relevant answers generated by the language model and grounded in your documents.

In the next section, we'll explore how to customize the skill to connect to your specific document collections and fine-tune its behavior for your use case.